# Managing Innovation Project

#### Installing packages

In [250]:
%%capture
#This could take a little while, but not to worry
#It won't display anything because I captured the output
import sys
!{sys.executable} -m pip install -r ../requirements.txt

#### Loading necessary packages:

In [238]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import plotly.express as px

import utils
import utils.numbers
from utils.data import load_sheet
from utils.sentiment import add_sentiment


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Loading the data

In [247]:
#Loads the excel table into a dictionary of DataFrames
sheet = load_sheet("../../.dat/Lego_subset_22_merge.xlsx")

Creating create a DataFrame where comments and submissions are joined, so that we can obtain comment count and sentiment

In [240]:
comments_df = sheet["ideas"].merge(
    sheet["comments"],
    how="left",
    suffixes=("_idea", "_comment"),
    on="submission_id"
)

In [241]:
comments_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 476 entries, 0 to 475
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   user_id_idea          476 non-null    int64         
 1   submission_id         476 non-null    int64         
 2   topic_alias_idea      476 non-null    object        
 3   title                 476 non-null    object        
 4   idea                  476 non-null    object        
 5   idea type             467 non-null    object        
 6   tags                  225 non-null    object        
 7   publish_date_idea     476 non-null    datetime64[ns]
 8   n_votes_idea          476 non-null    int64         
 9   expert_selected       476 non-null    bool          
 10  idea_experience       476 non-null    int64         
 11  user_id_comment       467 non-null    float64       
 12  topic_alias_comment   467 non-null    object        
 13  comment_id          

## Sentiment Analysis

We add sentiment scores based on the comments

In [242]:
sentiment_df = add_sentiment(comments_df, based_on="comment")

We summarize both sentiment polarity and sentiment subjectivity in a mean score for each submission.
I also add comment counts to each submission.
These newly added columns are then normalized, so that they are on the same scale

In [243]:
sentiment_summary = (
    sentiment_df[["submission_id", "sentiment_polarity", "sentiment_subjectivity"]]
    .groupby("submission_id")
    .mean()
)
comment_count = (
    comments_df[["submission_id", "comment"]]
    .groupby("submission_id")
    .count()
    .rename({"comment": "comment_count"}, axis=1)
)
df = (
    sheet["ideas"]
    .merge(sentiment_summary, on="submission_id")
    .merge(comment_count, on="submission_id")
)

Here are some histograms to help us investigate the distributions of the variables

In [244]:
px.histogram(df, x = "sentiment_polarity").show()
px.histogram(df, x = "sentiment_subjectivity").show()
px.histogram(df, x="n_votes").show()
px.histogram(df, x= "comment_count").show()

#### Interpretation:
 - We can clearly see that the overwhelming majority of comments is positive in sentiment,
 meaning that people don't usually comment when they have negative views about an idea.
 - The distribution of number of votes and comment count are very similar
 - Most posts get around 2 comments and 5 votes

We normalize these newly added columns, so that they are on the same scale and can be unified at will

In [245]:
df = utils.numbers.normalize_columns(
    df,
    ["sentiment_polarity", "sentiment_subjectivity", "n_votes", "comment_count"]
)

Some correlation analysis and plots about these variables

In [246]:
utils.numbers.corr_test(df, "sentiment_polarity", "sentiment_subjectivity")
utils.numbers.corr_test(df, "n_votes", "comment_count")
utils.numbers.corr_test(df, "sentiment_polarity", "n_votes")

SpearmanrResult(correlation=0.5884248250641151, pvalue=1.5048173222790123e-10)


SpearmanrResult(correlation=0.690806167784053, pvalue=1.3056054302164864e-16)


SpearmanrResult(correlation=0.15488009289790128, pvalue=0.1258390001765169)


### Interpretation:
 - We can see that sentiment subjectivity and polarity are highly positively correlated, meaning that the more positive a comment is, the more subjective language it uses.
 - Number of votes and number of comments is highly correlated, meaning that engagement is basically one variable
  - Number of votes is not correlated with sentiment polarity, meaning that higher average sentiment of comments does not necessarily mean more engagement

Since engagement is basically one variable we take the mean of the number of votes and number of comments, that way we can have a joint engagement variable 


In [115]:
df = df.assign(
    engagement=(df["n_votes"]+df["comment_count"])/2,
)

#### Top 10 ideas by engagement

In [117]:
most_popular = df.sort_values(by="engagement")["idea"].head(10)
for i, idea in enumerate(most_popular):
    print(f"{i+1}. - {idea}")

1. - Create Skylanders LEGO Figurs - the skylanders figurs should be a part of normal play sets and thereby both offer the physical play and play with the mini figure in a game.
2. - Combining a dungeon and dragons storytelling and game element to the LEGO play experience  having a LEGO play master (Dungeon master) that builds the story and set the quests for the rest of the people using the guidelines and framework set through the product
3. - A software or Guide for the creation or development for new system. in pmy pc when i have difent types of partes  i can creat a system an leather purched the pieces and recreat. and difent ways to mix difent systems."
4. - I built for several nights for the Unimog U400, a multi-function tool car. It is really a good way for us to build it. I feel so good to see the basic car machine operation in such a model. As an engineer, it is really better to build such model than only read it in book. Although I am not an Technic fans,  it is really intere

#### Top 10 ideas by mean sentiment of comments

In [120]:
most_loved = df.sort_values(by="sentiment_polarity")["idea"].head(10)
for i, idea in enumerate(most_loved):
    print(f"{i+1}. - {idea}")

1. - The successful business model for digital play has been found (but not yet perfected)...and the winner is: Skylanders! For those of you who dont know Skylanders here is a short intro: Skylanders is a video game for gaming consols (x-box,  ps3 etc). Along with the game you get a physical platform (RFID chip reader) and Skylander figures - placing one of the figures on the platform activates the figure in the game and lets you play with that character as long as the figure is placed upon the platform - and switching the figure obviously lets you play with a new character (with new skills,  new looks,  new persona etc). The game is basically running around in a virtual world fighting monsters and completing in-game mini games....much like LEGO universe. Looking at my nephew and his friends (age 7) this game has rapidly become a part of their daily routine. They bring the figures to each other so they can try out the new figures they have just bought/received.  In retail prices in DK 

#### Top 10 ideas by a balanced score of engagement and sentiment

In [121]:
df = df.assign(
    balanced_score = (df["engagement"]+df["sentiment_polarity"])/2,
)
balanced = df.sort_values(by="balanced_score")["idea"].head(10)
for i, idea in enumerate(balanced):
    print(f"{i+1}. - {idea}")

1. - The successful business model for digital play has been found (but not yet perfected)...and the winner is: Skylanders! For those of you who dont know Skylanders here is a short intro: Skylanders is a video game for gaming consols (x-box,  ps3 etc). Along with the game you get a physical platform (RFID chip reader) and Skylander figures - placing one of the figures on the platform activates the figure in the game and lets you play with that character as long as the figure is placed upon the platform - and switching the figure obviously lets you play with a new character (with new skills,  new looks,  new persona etc). The game is basically running around in a virtual world fighting monsters and completing in-game mini games....much like LEGO universe. Looking at my nephew and his friends (age 7) this game has rapidly become a part of their daily routine. They bring the figures to each other so they can try out the new figures they have just bought/received.  In retail prices in DK 

## Topic modelling
We use negative matrix factorization to look for some topics in the data.
As an arbitrary choice I decided to go with 5, but you can change that.
Words with a higher frequency than 20% in every document are filtered, so that they don't appear in every topic.

In [223]:
from utils.topic_modelling import NMFTopics, add_nmf_topics

nmf_model = NMFTopics(n_topics=5, cutoff_frequency=0.2).fit(df["idea"])

/home/kardosur/Documents/GitHub/minnov/GroupFive/environment/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning:

The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).



Here's the top 10 words in each topic ordered by how much they contribute to the topic

In [225]:
topics = nmf_model.get_topics(top_words=10)
for i, topic in enumerate(topics):
    print(i, " : ", topic)

0  :  {'theme': 0.5878434905790433, 'village': 0.47014692461518814, 'christmas': 0.4564955070781656, 'winter': 0.4497637054747745, 'city': 0.36409129115387573, 'product': 0.2793644680594245, 'inspire': 0.20395033630766726, 'european': 0.1999978208064295, 'year': 0.1881477036137403, 'good': 0.18239034234600934}
1  :  {'figure': 0.6465224014976043, 'game': 0.49147168947934006, 'new': 0.20197097714696255, 'character': 0.19335781666742538, 'physical': 0.16844602265773043, 'skylanders': 0.16644077842805274, 'create': 0.16582807297871702, 'robot': 0.1601375894770996, 'skylander': 0.14709423777695396, 'web': 0.13172145838215546}
2  :  {'child': 0.5093047215141081, 'minifigure': 0.4386307522170859, 'friend': 0.1828782629168556, 'app': 0.15184115667477294, 'great': 0.14138114273373073, 'family': 0.13213398171175994, 'design': 0.12439745901985333, 'etc': 0.12439700563000511, 'week': 0.12118598827998041, 'fun': 0.1187498591131233}
3  :  {'bag': 0.6175348422525017, 'open': 0.4889295367637017, 'sma

In [162]:
df = add_nmf_topics(df, "idea", nmf_model)

We plot the balanced scores of all ideas for each topic. It can be clearly seen that no topic is significantly more popular or more loved than any other.

In [191]:
px.box(df, x="idea_topic", y="balanced_score")

Here's a histogram of all the topics, you can clearly see that topic 4 has the most ideas submitted

In [222]:
px.histogram(df["idea_topic"])

On the  other hand it seems like experts seem to favour topic 2 quite drastically

In [196]:
count_expert_selected = (
    df[df["expert_selected"] == 1]
    .groupby("idea_topic")
    .count()["expert_selected"]
    .to_frame()
    .reset_index()
)
px.bar(count_expert_selected, x="idea_topic", y="expert_selected")

## Supervised learning

We transform the ideas to term frequency-inverse document frequency vectors, and then try to see
whether we can reliably classify if an idea is going to be selected by an expert or not.

In [226]:
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier

from utils.learn import evaluate_model

X = nmf_model.tf_idf.transform(df["idea"])
y = df["expert_selected"].tolist()

We test 2 models on the data:
 - DummyClassifier: A classifier that does not learn anything from the data, we use it to obtain chance level accuracy scores
 - RandomForest: An ensemble learning method that usually performs well in classification tasks and is quite robust against overfitting

In order to evaluate whether RandomForest performs any better than a the Dummy we conduct the following simulation 500 times:
 - We shuffle the data and divide it into a training and a testing set by a random split
 - We fit the models to the training part of the dataset
 - We test the models' accuracy on data it has not seen before, thereby seeing how much it generalises

In [235]:
n_sim=500
dummy_accuracy = evaluate_model(
    lambda: DummyClassifier(),
    X,y,
    n_sim=n_sim
)
rforest_accuracy = evaluate_model(
    RandomForestClassifier,
    X,y,
    n_sim=n_sim
)
evaluation_df = pd.DataFrame({
    "model": ["Dummy"]*n_sim+["RandomForest"]*n_sim,
    "accuracy": list(dummy_accuracy) + list(rforest_accuracy)
})

After the simulations we look at the box plot of accuracy scores to see if the two models perform significantly differently

In [236]:
px.box(evaluation_df, x="model", y="accuracy", points="all")

And we run a t-test to see whether the accuracy scores are significantly different from each other.

In [237]:
from scipy.stats import ttest_ind

ttest_ind(dummy_accuracy, rforest_accuracy)

Ttest_indResult(statistic=0.5969511782237217, pvalue=0.550675372754204)

#### Interpretation:
It can clearly be seen from the visualization that the models perform virtually identically.
Our t-test yields a p-value way above 0.05, meaning that there is a high probability that the small difference there is between the two distributions is simply due to random noise.

This essentially means that we did not manage to obtain a model that is capable of classifying an ideas as worthy for consideration based on word usage.